In [ ]:
from google.colab import files

# Upload the Kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!kaggle datasets download -d dommatap/obd-dataset
!kaggle datasets download -d cephasax/obdii-ds3

Dataset URL: https://www.kaggle.com/datasets/cephasax/obdii-ds3
License(s): CC0-1.0
 95% 7.00M/7.39M [00:00<00:00, 13.4MB/s]
100% 7.39M/7.39M [00:00<00:00, 9.99MB/s]


In [ ]:
#!unzip /content/obd-dataset.zip
!unzip /content/obdii-ds3.zip

Archive:  /content/obdii-ds3.zip
  inflating: exp1_14drivers_14cars_dailyRoutes.csv  
  inflating: exp1_14drivers_14cars_dailyRoutes.xlsx  
  inflating: exp2_19drivers_1car_1route.csv  
  inflating: exp2_19drivers_1car_1route.xlsx  
  inflating: exp3_4drivers_1car_1route.csv  
  inflating: exp3_4drivers_1car_1route.xlsx  


In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report , confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
data = pd.read_csv('/content/exp1_14drivers_14cars_dailyRoutes.csv')
data.head()

<ipython-input-6-387a1899f929>:1: DtypeWarning: Columns (1,2,4,5,6,9,10,14,15,16,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/exp1_14drivers_14cars_dailyRoutes.csv')


,TIMESTAMP,MARK,MODEL,CAR_YEAR,ENGINE_POWER,AUTOMATIC,VEHICLE_ID,BAROMETRIC_PRESSURE(KPA),ENGINE_COOLANT_TEMP,FUEL_LEVEL,...,THROTTLE_POS,DTC_NUMBER,TROUBLE_CODES,TIMING_ADVANCE,EQUIV_RATIO,MIN,HOURS,DAYS_OF_WEEK,MONTHS,YEAR
0,1.502903e+12,chevrolet,agile,2011.0,"1,4",n,car1,100.0,80.0,"48,60%",...,25%,MIL is OFF0 codes,NaN,"56,9%","1,0%",13.0,16.0,2.0,8.0,2017.0
1,1.502903e+12,chevrolet,agile,2011.0,"1,4",n,car1,100.0,80.0,"48,60%",...,25%,MIL is OFF0 codes,NaN,"56,5%","1,0%",13.0,16.0,2.0,8.0,2017.0
2,1.502903e+12,chevrolet,agile,2011.0,"1,4",n,car1,100.0,80.0,"48,60%",...,25%,MIL is OFF0 codes,NaN,"57,3%","1,0%",13.0,16.0,2.0,8.0,2017.0
3,1.502903e+12,chevrolet,agile,2011.0,"1,4",n,car1,100.0,80.0,"48,60%",...,25%,MIL is OFF0 codes,NaN,"56,5%","1,0%",13.0,16.0,2.0,8.0,2017.0
4,1.502903e+12,chevrolet,agile,2011.0,"1,4",n,car1,100.0,80.0,"48,60%",...,25%,MIL is OFF0 codes,NaN,"56,9%","1,0%",13.0,16.0,2.0,8.0,2017.0


In [ ]:
data.shape

(60439, 33)

# Data Preprocessing

In [ ]:
# Replacing NaN values with 'normal' based on the condition
data.loc[(data['DTC_NUMBER'] == 'MIL is OFF0 codes') & (data['TROUBLE_CODES'].isna()), 'TROUBLE_CODES'] = 'normal'

In [ ]:
data.dropna(subset=['TROUBLE_CODES'], inplace= True)

In [ ]:
!pip install dtale

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 57.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 79.4 MB/s eta 0:00:00
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-py3-none-any.whl size=62566 sha256=94e101c

In [ ]:
import dtale
import dtale.app as dtale_app

dtale_app.USE_COLAB = True

# Start the dtale web app
dtale = dtale.show(data_2)

dtale

https://jdms29is5us-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/2

In [ ]:
d_columns= ['TIMESTAMP','DTC_NUMBER','AUTOMATIC','EQUIV_RATIO','ENGINE_RUNTIME','FUEL_TYPE','SHORT TERM FUEL TRIM BANK 2',
    'FUEL_PRESSURE','LONG TERM FUEL TRIM BANK 2','MAF','AMBIENT_AIR_TEMP','FUEL_LEVEL','BAROMETRIC_PRESSURE(KPA)',
    'VEHICLE_ID','MODEL','YEAR','MARK','MONTHS','DAYS_OF_WEEK','MIN','VEHICLE_ID','INTAKE_MANIFOLD_PRESSURE','CAR_YEAR']
data=data.drop(d_columns, axis=1)

In [ ]:
data['SHORT TERM FUEL TRIM BANK 1']= data['SHORT TERM FUEL TRIM BANK 1'].str.split("%").str.get(0).astype(float)

In [ ]:
data['ENGINE_LOAD']= data['ENGINE_LOAD'].str.split("%").str.get(0)
data['ENGINE_LOAD'] = data['ENGINE_LOAD'].str.replace(',', '.').astype(float)

In [ ]:
data['THROTTLE_POS']= data['THROTTLE_POS'].str.split("%").str.get(0).astype(float)

In [ ]:
data['TIMING_ADVANCE']= data['TIMING_ADVANCE'].str.split("%").str.get(0)
data['TIMING_ADVANCE'] = data['TIMING_ADVANCE'].str.replace(',', '.').astype(float)

In [ ]:
data['ENGINE_POWER']=data['ENGINE_POWER'].replace({'1,6':1.6,'1,8':1.8,'1,4':1.4})
data['ENGINE_POWER'] = data['ENGINE_POWER'].astype(float)

In [ ]:
mean_value = data['SHORT TERM FUEL TRIM BANK 1'].mean()
data['SHORT TERM FUEL TRIM BANK 1'].fillna(mean_value, inplace=True)

mean_value = data['ENGINE_POWER'].mean()
data['ENGINE_POWER'].fillna(mean_value, inplace=True)

mean_value = data['ENGINE_LOAD'].mean()
data['ENGINE_LOAD'].fillna(mean_value, inplace=True)

mean_value = data['THROTTLE_POS'].mean()
data['THROTTLE_POS'].fillna(mean_value, inplace=True)

mean_value = data['TIMING_ADVANCE'].mean()
data['TIMING_ADVANCE'].fillna(mean_value, inplace=True)

# Assuming df is your DataFrame
#data['TROUBLE_CODES'] = data['TROUBLE_CODES'].fillna('normal')

In [ ]:
data = data.interpolate()

In [ ]:
data.columns

Index(['ENGINE_POWER', 'ENGINE_COOLANT_TEMP', 'ENGINE_LOAD', 'ENGINE_RPM',
       'AIR_INTAKE_TEMP', 'SPEED', 'SHORT TERM FUEL TRIM BANK 1',
       'THROTTLE_POS', 'TROUBLE_CODES', 'TIMING_ADVANCE', 'HOURS'],
      dtype='object')

In [ ]:
data.isna().sum()

ENGINE_POWER                   0
ENGINE_COOLANT_TEMP            0
ENGINE_LOAD                    0
ENGINE_RPM                     0
AIR_INTAKE_TEMP                0
SPEED                          0
SHORT TERM FUEL TRIM BANK 1    0
THROTTLE_POS                   0
TROUBLE_CODES                  0
TIMING_ADVANCE                 0
HOURS                          0
dtype: int64

In [ ]:
data.head()

,ENGINE_POWER,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,AIR_INTAKE_TEMP,SPEED,SHORT TERM FUEL TRIM BANK 1,THROTTLE_POS,TROUBLE_CODES,TIMING_ADVANCE,HOURS
0,1.4,80.0,33.3,1009.0,59.0,0.0,-35.117014,25.0,normal,56.9,16.0
1,1.4,80.0,32.5,1003.0,59.0,0.0,-35.117014,25.0,normal,56.5,16.0
2,1.4,80.0,32.9,995.0,59.0,0.0,-35.117014,25.0,normal,57.3,16.0
3,1.4,80.0,32.5,1004.0,60.0,0.0,-35.117014,25.0,normal,56.5,16.0
4,1.4,80.0,32.9,1005.0,60.0,0.0,-35.117014,25.0,normal,56.9,16.0


In [ ]:
#data['TROUBLE_CODES'].unique()

In [ ]:
data_2 = data.copy()

In [ ]:
data_3 = data.copy()

In [ ]:
# Assuming 'Predicted' column contains values 1 or 2
filtered_data = data[(data['TROUBLE_CODES'] == "P0079P2004P3000")]
filtered_data.head()

,ENGINE_POWER,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,AIR_INTAKE_TEMP,SPEED,SHORT TERM FUEL TRIM BANK 1,THROTTLE_POS,TROUBLE_CODES,TIMING_ADVANCE,HOURS
45310,1.4,55.0,40.439026,2424.0,29.0,49.0,2.0,26.000000,P0079P2004P3000,63.9,10.0
45354,1.4,91.0,86.300000,2725.0,34.0,68.0,2.0,17.971916,P0079P2004P3000,58.0,10.0
45554,1.4,62.0,40.439026,819.0,28.0,0.0,-10.0,15.000000,P0079P2004P3000,53.7,11.0
45568,1.4,83.0,44.700000,2866.0,28.0,93.0,-6.0,24.000000,P0079P2004P3000,64.7,11.0
45679,1.4,92.0,36.900000,783.0,44.0,0.0,-4.0,15.000000,P0079P2004P3000,52.9,11.0


# ML Models

Binary Classification Processing


In [ ]:
# Count the occurrences of each value in the 'TROUBLE_CODES' column
counts = data['TROUBLE_CODES'].value_counts()

print(counts)

TROUBLE_CODES
normal             35214
P0133               6070
C0300               5673
P0079P2004P3000       48
P0079C1004P3000       30
P0078U1004P3000       29
P007EP2036P18D0       20
P007EP2036P18E0       18
P0078B0004P3000       12
P007EP2036P18F0        9
P0079P1004P3000        5
P007FP2036P18E0        5
P007FP2036P18D0        3
P007FP2036P18F0        3
Name: count, dtype: int64


In [ ]:
# Encode the 'TROUBLE_CODES' column
data['T_C'] = data['TROUBLE_CODES'].apply(lambda x: 0 if x == 'normal' else 1)

In [ ]:
# Count the occurrences of each value in the 'TROUBLE_CODES' column
counts = data['T_C'].value_counts()

print(counts)

T_C
0    35214
1    11925
Name: count, dtype: int64


In [ ]:
#under-sampling
normal_indices = data[data['TROUBLE_CODES'] == 'normal'].sample(n=23214, random_state=42).index
data =data.drop(normal_indices)

In [ ]:
# Count the occurrences of each value in the 'TROUBLE_CODES' column
counts = data['T_C'].value_counts()

print(counts)

T_C
0    12000
1    11925
Name: count, dtype: int64


Binary Classification Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Features and target variable
X = data.drop(['T_C' ,'TROUBLE_CODES', 'HOURS'] , axis=1)
y = data['T_C']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a binary classification model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))




Accuracy: 0.9832810867293625
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      2387
           1       0.99      0.98      0.98      2398

    accuracy                           0.98      4785
   macro avg       0.98      0.98      0.98      4785
weighted avg       0.98      0.98      0.98      4785



In [ ]:
def print_classification_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        #print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")

In [ ]:
# Evaluate the classifier on training data
print_classification_score(model, X_train, y_train, X_test,y_test, train=True)
# Evaluate the classifier on test data
print_classification_score(model, X_train, y_train, X_test,y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                0       1  accuracy  macro avg  weighted avg
precision     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0       1.0        1.0           1.0
support    9613.0  9527.0       1.0    19140.0       19140.0
_______________________________________________
Test Result:
Accuracy Score: 98.33%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.978432     0.988206  0.983281     0.983319      0.983330
recall        0.988270     0.978315  0.983281     0.983293      0.983281
f1-score      0.983326     0.983236  0.983281     0.983281      0.983281
support    2387.000000  2398.000000  0.983281  4785.000000   4785.000000
_______________________________________________


In [ ]:
import pickle
filename = 'binary_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('/content/binary_model.sav', 'rb'))

# Data Generation Model


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Separate features and targets
features = data_2.drop(columns=['TROUBLE_CODES', 'HOURS'])
trouble_codes = data_2['TROUBLE_CODES']
hours = data_2['HOURS']

# Normalize continuous features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# One-hot encode TROUBLE_CODES
encoder = OneHotEncoder(sparse_output=False)
trouble_codes_encoded = encoder.fit_transform(trouble_codes.values.reshape(-1, 1))

# Combine features with targets
combined_data = np.hstack((features_scaled, trouble_codes_encoded, hours.values.reshape(-1, 1)))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Input, concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Define CGAN architecture
def build_generator(latent_dim, num_classes, num_features):
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(num_classes,))
    merged_input = concatenate([noise_input, label_input])

    x = Dense(128)(merged_input)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(num_features)(x)

    model = Model([noise_input, label_input], x)
    return model

def build_discriminator(num_features, num_classes):
    data_input = Input(shape=(num_features,))
    label_input = Input(shape=(num_classes,))
    merged_input = concatenate([data_input, label_input])

    x = Dense(512)(merged_input)
    x = LeakyReLU(alpha=0.01)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.01)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model([data_input, label_input], output)
    return model

# Hyperparameters
latent_dim = 100
num_classes = trouble_codes_encoded.shape[1]
num_features = combined_data.shape[1]

# Build and compile the discriminator
discriminator = build_discriminator(num_features, num_classes)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim, num_classes, num_features)

# Build and compile the CGAN
noise_input = Input(shape=(latent_dim,))
label_input = Input(shape=(num_classes,))
gen_data = generator([noise_input, label_input])
discriminator.trainable = False
validity = discriminator([gen_data, label_input])

cgan = Model([noise_input, label_input], validity)
cgan.compile(loss='binary_crossentropy', optimizer=Adam())

# Training the CGAN
def train_cgan(generator, discriminator, cgan, data, labels, epochs, batch_size, latent_dim):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_data = data[idx]
        real_labels = labels[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_data = generator.predict([noise, real_labels])

        d_loss_real = discriminator.train_on_batch([real_data, real_labels], valid)
        d_loss_fake = discriminator.train_on_batch([gen_data, real_labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = cgan.train_on_batch([noise, real_labels], valid)

        # Print the progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

train_cgan(generator, discriminator, cgan, combined_data, trouble_codes_encoded, epochs=10000, batch_size=32, latent_dim=latent_dim)



Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step
5100 [D loss: 0.19814449548721313 | D accuracy: 90.625] [G loss: 3.627584457397461]
1/1 [==============================] - 0s 35ms/step
5200 [D loss: 0.2946159839630127 | D accuracy: 89.0625] [G loss: 2.909148693084717]
1/1 [==============================] - 0s 26ms/step
5300 [D loss: 0.1433734968304634 | D accuracy: 90.625] [G loss: 6.095149517059326]
1/1 [==============================] - 0s 27ms/step
5400 [D loss: 0.11327933520078659 | D accuracy: 96.875] [G loss: 4.7787065505981445]
1/1 [==============================] - 0s 31ms/step
5500 [D loss: 0.12006519362330437 | D accuracy: 95.3125] [G loss: 3.900688409805298]
1/1 [==============================] - 0s 29ms/step
5600 [D loss: 0.5102655440568924 | D accuracy: 84.375] [G loss: 4.1193366050720215]
1/1 [==============================] - 0s 29ms/step
5700 [D loss: 0.06631069630384445 | D accuracy: 100.0] [G loss: 5.19621753692626

In [ ]:
# Generate new data
def generate_synthetic_data(generator, num_samples, latent_dim, num_classes):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    # Sample labels from the available categories, not random integers
    sampled_labels = np.random.choice(encoder.categories_[0], num_samples)
    sampled_labels = encoder.transform(sampled_labels.reshape(-1, 1))
    gen_data = generator.predict([noise, sampled_labels])
    return gen_data, sampled_labels

synthetic_data, synthetic_labels = generate_synthetic_data(generator, 2600, latent_dim, num_classes)

# Combine synthetic data with original data
synthetic_trouble_codes = encoder.inverse_transform(synthetic_labels)
synthetic_hours = synthetic_data[:, -1]  # Adjust this line as needed if you had normalization applied earlier
synthetic_features = synthetic_data[:, :-num_classes-1]

# Create a synthetic DataFrame
synthetic_df = pd.DataFrame(scaler.inverse_transform(synthetic_features), columns=features.columns)
# Flatten the synthetic_trouble_codes array before assigning it to the DataFrame.
synthetic_df['TROUBLE_CODES'] = synthetic_trouble_codes.flatten()
synthetic_df['HOURS'] = synthetic_hours

82/82 [==============================] - 0s 2ms/step


In [ ]:
synthetic_df['TROUBLE_CODES'].value_counts()

TROUBLE_CODES
P007EP2036P18E0    217
P007FP2036P18E0    202
P0133              201
normal             200
P0079C1004P3000    193
P0078B0004P3000    193
C0300              189
P007FP2036P18D0    178
P0079P2004P3000    178
P0078U1004P3000    177
P007EP2036P18D0    172
P007EP2036P18F0    169
P007FP2036P18F0    166
P0079P1004P3000    165
Name: count, dtype: int64

In [ ]:
# Remove 1000 'normal' values randomly
normal_indices = synthetic_df[synthetic_df['TROUBLE_CODES'] == 'normal'].sample(n=200, random_state=42).index
synthetic_df =synthetic_df.drop(normal_indices)

# Remove 300 'P0133' values randomly
P0133_indices = synthetic_df[synthetic_df['TROUBLE_CODES'] == 'P0133'].sample(n=201, random_state=42).index
synthetic_df = synthetic_df.drop(P0133_indices)

# Remove 200 'C0300' values randomly
C0300_indices = synthetic_df[synthetic_df['TROUBLE_CODES'] == 'C0300'].sample(n=189, random_state=42).index
synthetic_df = synthetic_df.drop(C0300_indices)

# Reset the index after dropping rows
synthetic_df.reset_index(drop=True, inplace=True)

print(synthetic_df['TROUBLE_CODES'].value_counts())

TROUBLE_CODES
P007EP2036P18E0    217
P007FP2036P18E0    202
P0079C1004P3000    193
P0078B0004P3000    193
P007FP2036P18D0    178
P0079P2004P3000    178
P0078U1004P3000    177
P007EP2036P18D0    172
P007EP2036P18F0    169
P007FP2036P18F0    166
P0079P1004P3000    165
Name: count, dtype: int64


In [ ]:
# Remove 1000 'normal' values randomly
normal_indices = data_2[data_2['TROUBLE_CODES'] == 'normal'].sample(n=35214, random_state=42).index
data_2 = data_2.drop(normal_indices)

# Reset the index after dropping rows
data_2.reset_index(drop=True, inplace=True)

print(data_2['TROUBLE_CODES'].value_counts())

TROUBLE_CODES
P0133              6070
C0300              5673
P0079P2004P3000      48
P0079C1004P3000      30
P0078U1004P3000      29
P007EP2036P18D0      20
P007EP2036P18E0      18
P0078B0004P3000      12
P007EP2036P18F0       9
P007FP2036P18E0       5
P0079P1004P3000       5
P007FP2036P18D0       3
P007FP2036P18F0       3
Name: count, dtype: int64


In [ ]:
print(data_2['TROUBLE_CODES'].value_counts())

TROUBLE_CODES
P0133              6070
C0300              5673
P0079P2004P3000      48
P0079C1004P3000      30
P0078U1004P3000      29
P007EP2036P18D0      20
P007EP2036P18E0      18
P0078B0004P3000      12
P007EP2036P18F0       9
P0079P1004P3000       5
P007FP2036P18E0       5
P007FP2036P18D0       3
P007FP2036P18F0       3
Name: count, dtype: int64


In [ ]:
df=  pd.concat([data_2,synthetic_df], ignore_index=True)

In [ ]:
df['TROUBLE_CODES'].value_counts()

TROUBLE_CODES
P0133              6070
C0300              5673
P007EP2036P18E0     235
P0079P2004P3000     226
P0079C1004P3000     223
P007FP2036P18E0     207
P0078U1004P3000     206
P0078B0004P3000     205
P007EP2036P18D0     192
P007FP2036P18D0     181
P007EP2036P18F0     178
P0079P1004P3000     170
P007FP2036P18F0     169
Name: count, dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report , confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Define the label mapping dictionary
label_mapp = {
    'P0133': 0,
    'C0300': 1,
    'P0079P2004P3000': 2,
    'P0078U1004P3000': 3,
    'P0079C1004P3000': 4,
    'P007EP2036P18F0': 5,
    'P007EP2036P18D0': 6,
    'P007FP2036P18D0': 7,
    'P0079P1004P3000': 8,
    'P007EP2036P18E0':9,
    'P007FP2036P18E0':10,
    'P0078B0004P3000':11,
    'P007FP2036P18F0':12
}

# Label the column using the label mapping
df['TROUBLE_CODES'] = df['TROUBLE_CODES'].map(label_mapp)


In [ ]:
# Split the data into features and targets
X = df.drop(columns=['TROUBLE_CODES', 'HOURS'])
y_classification = df['TROUBLE_CODES']


# Split the data into training and testing sets
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y_classification, test_size=0.20, random_state=42)


In [ ]:
random_forest = RandomForestClassifier(n_estimators=50,max_depth=7)

random_forest.fit(X_train_class, y_train_class)
y_pred_random_forest = random_forest.predict(X_test_class)

In [ ]:
def print_classification_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        #print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        #print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")


In [ ]:
# Evaluate the classifier on training data
print_classification_score(random_forest, X_train_class, y_train_class, X_test_class, y_test_class, train=True)
# Evaluate the classifier on test data
print_classification_score(random_forest, X_train_class, y_train_class, X_test_class, y_test_class, train=False)


Train Result:
Accuracy Score: 91.01%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1           2           3           4  \
precision     0.999588     0.945793    0.424658    0.516779    0.864198   
recall        1.000000     0.999558    0.656085    0.475309    0.393258   
f1-score      0.999794     0.971932    0.515593    0.495177    0.540541   
support    4852.000000  4521.000000  189.000000  162.000000  178.000000   

                    5           6           7           8           9  \
precision    0.591837    0.818182    0.625000    0.687500    0.306667   
recall       0.405594    0.233766    0.493421    0.235714    0.715026   
f1-score     0.481328    0.363636    0.551471    0.351064    0.429238   
support    143.000000  154.000000  152.000000  140.000000  193.000000   

                   10          11          12  accuracy     macro avg  \
precision    0.568182    0.882353    0.862745  0.910118      0.699498   
reca

In [ ]:
import pickle
filename = 'Multi_Classification_Model2.sav'
pickle.dump(random_forest, open(filename, 'wb'))

regression model

In [ ]:
# Define the label mapping dictionary
label_mapp = {
    'normal':0,
    'P0133': 1,
    'C0300': 2,
    'P0079P2004P3000': 3,
    'P0078U1004P3000': 4,
    'P0079C1004P3000': 5,
    'P007EP2036P18F0': 6,
    'P007EP2036P18D0': 7,
    'P007FP2036P18D0': 8,
    'P0079P1004P3000': 9,
    'P007EP2036P18E0':10,
    'P007FP2036P18E0':11,
    'P0078B0004P3000':12,
    'P007FP2036P18F0':13
}

# Label the column using the label mapping
data_3['TROUBLE_CODES'] = data_3['TROUBLE_CODES'].map(label_mapp)

In [ ]:
# Split the data into features and targets
X = data_3.drop(columns=['HOURS', 'TROUBLE_CODES'])
y_regression = data_3['HOURS']

# Split the data into training and testing sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.2, random_state=42)

In [ ]:
extra_trees_model = ExtraTreesRegressor(max_depth=25, n_estimators=30)
extra_trees_model.fit(X_train_reg, y_train_reg)
y_pred_extra_trees = extra_trees_model.predict(X_test_reg)

In [ ]:
def print_regression_score(reg, x_train, y_train, x_test, y_test, train=True):
    if train:
        pred = reg.predict(x_train)
        mae = mean_absolute_error(y_train, pred)
        mse = mean_squared_error(y_train, pred)
        r2 = r2_score(y_train, pred)

        print("Train Result:\n================================================")
        print(f"Mean Absolute Error: {mae:.2f}")
        print(f"Mean Squared Error: {mse:.2f}")
        print(f"R-squared: {r2:.4f}")

    elif not train:
        pred = reg.predict(x_test)
        mae = mean_absolute_error(y_test, pred)
        mse = mean_squared_error(y_test, pred)
        r2 = r2_score(y_test, pred)

        print("Test Result:\n================================================")
        print(f"Mean Absolute Error: {mae:.2f}")
        print(f"Mean Squared Error: {mse:.2f}")
        print(f"R-squared: {r2:.4f}")

In [ ]:
# Evaluate the regressor on training data
print_regression_score(extra_trees_model, X_train_reg, y_train_reg, X_test_reg, y_test_reg, train=True)

# Evaluate the regressor on test data
print_regression_score(extra_trees_model, X_train_reg, y_train_reg, X_test_reg, y_test_reg, train=False)

Train Result:
Mean Absolute Error: 0.22
Mean Squared Error: 0.23
R-squared: 0.9948
Test Result:
Mean Absolute Error: 2.14
Mean Squared Error: 13.84
R-squared: 0.6738


In [ ]:
import pickle
filename = 'Regression_Hours_Model.sav'
pickle.dump(extra_trees_model, open(filename, 'wb'))

In [ ]:
!pip install pycaret

binary class models

In [ ]:
Data = data.drop(['TROUBLE_CODES'] , axis=1)

In [ ]:
from pycaret.classification import *
s = setup(Data, target = 'T_C', session_id = 123)

,Description,Value
0,Session id,123
1,Target,T_C
2,Target type,Binary
3,Original data shape,"(23925, 11)"
4,Transformed data shape,"(23925, 11)"
5,Transformed train set shape,"(16747, 11)"
6,Transformed test set shape,"(7178, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9879,0.9993,0.9869,0.9887,0.9878,0.9758,0.9758,1.0510
rf,Random Forest Classifier,0.9872,0.9993,0.9835,0.9908,0.9871,0.9744,0.9745,2.3930
xgboost,Extreme Gradient Boosting,0.9871,0.9993,0.9863,0.9878,0.9871,0.9742,0.9742,0.2520
lightgbm,Light Gradient Boosting Machine,0.9871,0.9992,0.9844,0.9897,0.9870,0.9742,0.9742,4.5500
dt,Decision Tree Classifier,0.9796,0.9796,0.9805,0.9786,0.9795,0.9592,0.9592,0.1740
gbc,Gradient Boosting Classifier,0.9789,0.9977,0.9788,0.9788,0.9788,0.9577,0.9577,2.2080
ada,Ada Boost Classifier,0.9650,0.9949,0.9753,0.9555,0.9653,0.9300,0.9303,1.2130
nb,Naive Bayes,0.9326,0.9775,0.9729,0.9000,0.9350,0.8652,0.8681,0.0770
qda,Quadratic Discriminant Analysis,0.9301,0.9836,0.9584,0.9066,0.9318,0.8602,0.8616,0.0820
knn,K Neighbors Classifier,0.9235,0.9720,0.9547,0.8983,0.9256,0.8470,0.8488,0.3590


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
# functional API
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

multiclass models

In [ ]:
from pycaret.classification import *
s = setup(df, target = 'TROUBLE_CODES', session_id = 123)

,Description,Value
0,Session id,123
1,Target,TROUBLE_CODES
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12"
4,Original data shape,"(13927, 11)"
5,Transformed data shape,"(13927, 11)"
6,Transformed train set shape,"(9748, 11)"
7,Transformed test set shape,"(4179, 11)"
8,Numeric features,10
9,Preprocess,True


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8617,0.0000,0.8617,0.8618,0.8612,0.7845,0.7847,22.3350
dt,Decision Tree Classifier,0.8610,0.9293,0.8610,0.8606,0.8605,0.7834,0.7835,0.0820
rf,Random Forest Classifier,0.8607,0.9889,0.8607,0.8579,0.8589,0.7826,0.7827,2.0150
xgboost,Extreme Gradient Boosting,0.8606,0.9897,0.8606,0.8603,0.8601,0.7827,0.7828,2.3560
lightgbm,Light Gradient Boosting Machine,0.8606,0.9896,0.8606,0.8600,0.8599,0.7828,0.7829,18.9550
et,Extra Trees Classifier,0.8592,0.9887,0.8592,0.8443,0.8510,0.7782,0.7786,0.7300
lda,Linear Discriminant Analysis,0.8520,0.0000,0.8520,0.8290,0.8377,0.7650,0.7661,0.0520
lr,Logistic Regression,0.8509,0.0000,0.8509,0.8066,0.8256,0.7597,0.7625,4.6400
nb,Naive Bayes,0.8497,0.9859,0.8497,0.8459,0.8445,0.7653,0.7671,0.0570
knn,K Neighbors Classifier,0.8475,0.9353,0.8475,0.8002,0.8210,0.7533,0.7563,0.1360


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

save model

In [ ]:
! pip install streamlit openai langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

%%writefile app.py
import streamlit as st
import pickle
import numpy as np
from openai import OpenAI
from langchain.llms import OpenAI as LangChainOpenAI
from langchain import PromptTemplate
import os

# Load the models from the files
binary_model = pickle.load(open('/content/binary_model.sav', 'rb'))
multi_model = pickle.load(open('/content/Multi_Classification_Model2.sav', 'rb'))
regression_model = pickle.load(open('/content/Regression_Hours_Model.sav', 'rb'))

# Set the OpenAI API key
api_key ='your_api_key'
client = OpenAI(api_key=api_key,max_retries=5,timeout=500)

# Function to generate response using OpenAI GPT-3

def generate_response(predict, hours):
    # Prompt
    template = '''As an expert in automotive diagnostics,
                 I need detailed information about {predict} Trouble Code, including their meanings, reasons for appearance,
                 and recommended treatments. Additionally,
                 I would like insights into how this {predict} Trouble Codes may impact the car's performance and safety.
                 Specifically, provide information on potential problems in the car associated with these codes {predict},
                 the recommended treatments or repairs, and any safety concerns.
                 Also, include details on the estimated time remaining before a car might encounter severe issues or
                 safety risks if the Trouble Codes are not addressed. The estimated time remaining is {hours} hours.'''


    prompt_query = template.format(predict=predict, hours=hours)
    messages = [{"role": "user", "content": prompt_query}]

    completion_args = {
            "model": "gpt-4o",
            "messages": messages}


    completion =client.chat.completions.create(**completion_args)
    return completion.choices[0].message.content


# Function to get user input and make a prediction
def predict():

    st.title("✨ TC Model Prediction")
    st.sidebar.subheader('User Input')

    # Get user input for each feature using number inputs in the sidebar
    engine_power = st.sidebar.number_input("Engine Power", min_value=1.0, max_value=3.0, step=0.1)
    engine_coolant_temp = st.sidebar.number_input("Engine Coolant Temp", min_value=0.0, max_value=110.0, step=1.0)
    engine_load = st.sidebar.number_input("Engine Load", min_value=0.0, max_value=100.0, step=0.01)
    engine_rpm = st.sidebar.number_input("Engine RPM", min_value=0.0, max_value=5000.0, step=10.0)
    air_intake_temp = st.sidebar.number_input("Air Intake Temp", min_value=-10.0, max_value=800.0, step=1.0)
    speed = st.sidebar.number_input("Speed", min_value=0.0, max_value=500.0, step=1.0)
    short_term_fuel_trim = st.sidebar.number_input("Short Term Fuel Trim Bank 1", min_value=-100.0, max_value=100.0, step=0.01)
    throttle_pos = st.sidebar.number_input("Throttle Position", min_value=0.0, max_value=100.0, step=1.0)
    timing_advance = st.sidebar.number_input("Timing Advance", min_value=0.0, max_value=100.0, step=1.0)

    # Add a horizontal line with a white background color
    st.sidebar.markdown("<hr class='sidebar-hr'>", unsafe_allow_html=True)

    # Create a NumPy array with the user input
    input_data = np.array([
        [engine_power, engine_coolant_temp, engine_load, engine_rpm,
         air_intake_temp, speed, short_term_fuel_trim, throttle_pos, timing_advance]
    ])

    # Make the binary classification prediction
    binary_prediction = binary_model.predict(input_data)[0]

    if binary_prediction == 0:
        st.subheader('Prediction: Normal')
    else:
        st.subheader('Prediction: Problem Detected')

        # Make the multi-class classification prediction
        multi_prediction = multi_model.predict(input_data)[0]
        trouble_codes = np.array(['P0133', 'C0300', 'P0079P2004P3000', 'P0078U1004P3000',
                                  'P0079C1004P3000', 'P007EP2036P18F0', 'P007EP2036P18D0',
                                  'P007FP2036P18D0', 'P0079P1004P3000', 'P007EP2036P18E0',
                                  'P007FP2036P18E0', 'P0078B0004P3000', 'P007FP2036P18F0'])
        trouble_code = trouble_codes[multi_prediction]
        st.subheader(f'Trouble Code: {trouble_code}')

        # Predict the number of hours for the car
        hours_prediction = regression_model.predict(input_data)[0]
        st.subheader(f'Estimated Time Remaining: {hours_prediction:.2f} hours')

        # Generate and display the OpenAI GPT-3 response
        response = generate_response(trouble_code, hours_prediction)
        st.subheader('OpenAI GPT-4o Response')
        st.write(response)

# Run the app
if __name__ == "__main__":
    predict()


Overwriting app.py


In [ ]:
!pip install -q streamlit

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.264s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.225.110


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.049s
your url is: https://mighty-jeans-send.loca.lt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
